In [1]:

import pandas as pd
import tsfresh
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


In [3]:

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00501/PRSA2017_Data_20130301-20170228.zip"
r = urlopen(url)
zf = ZipFile(BytesIO(r.read()))

df = pd.DataFrame()
for file in zf.infolist():
    if file.filename.endswith('.csv'):
        df = df.append(pd.read_csv(zf.open(file)))

df['timestamp'] = pd.to_datetime(df[["year", "month", "day", "hour"]])
df.drop(columns=['No'], inplace=True)
df.sort_values(by=['timestamp', 'station']).head(20)


C:\Users\haman\AppData\Local\Temp\ipykernel_14400\581080754.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(zf.open(file)))
C:\Users\haman\AppData\Local\Temp\ipykernel_14400\581080754.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(zf.open(file)))
C:\Users\haman\AppData\Local\Temp\ipykernel_14400\581080754.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(zf.open(file)))
C:\Users\haman\AppData\Local\Temp\ipykernel_14400\581080754.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(zf.open(file)))
C:\Users\haman\AppData\L

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station,timestamp
0,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,2013-03-01 00:00:00
0,2013,3,1,0,3.0,6.0,13.0,7.0,300.0,85.0,-2.3,1020.8,-19.7,0.0,E,0.5,Changping,2013-03-01 00:00:00
0,2013,3,1,0,4.0,4.0,3.0,NaN,200.0,82.0,-2.3,1020.8,-19.7,0.0,E,0.5,Dingling,2013-03-01 00:00:00
0,2013,3,1,0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Dongsi,2013-03-01 00:00:00
0,2013,3,1,0,4.0,4.0,14.0,20.0,300.0,69.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Guanyuan,2013-03-01 00:00:00
0,2013,3,1,0,6.0,18.0,5.0,NaN,800.0,88.0,0.1,1021.1,-18.6,0.0,NW,4.4,Gucheng,2013-03-01 00:00:00
0,2013,3,1,0,7.0,7.0,3.0,2.0,100.0,91.0,-2.3,1020.3,-20.7,0.0,WNW,3.1,Huairou,2013-03-01 00:00:00
0,2013,3,1,0,5.0,14.0,4.0,12.0,200.0,85.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Nongzhanguan,2013-03-01 00:00:00
0,2013,3,1,0,3.0,6.0,3.0,8.0,300.0,44.0,-0.9,1025.8,-20.5,0.0,NW,9.3,Shunyi,2013-03-01 00:00:00
0,2013,3,1,0,6.0,6.0,4.0,8.0,300.0,81.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Tiantan,2013-03-01 00:00:00


In [4]:
df['station'].value_counts()


Aotizhongxin     35064
Changping        35064
Dingling         35064
Dongsi           35064
Guanyuan         35064
Gucheng          35064
Huairou          35064
Nongzhanguan     35064
Shunyi           35064
Tiantan          35064
Wanliu           35064
Wanshouxigong    35064
Name: station, dtype: int64

In [5]:

df.isnull().sum()


year             0
month            0
day              0
hour             0
PM2.5         8739
PM10          6449
SO2           9021
NO2          12116
CO           20701
O3           13277
TEMP           398
PRES           393
DEWP           403
RAIN           390
wd            1822
WSPM           318
station          0
timestamp        0
dtype: int64

In [6]:
df.fillna(df.mean(), inplace=True)

C:\Users\haman\AppData\Local\Temp\ipykernel_14400\820435583.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df.fillna(df.mean(), inplace=True)
C:\Users\haman\AppData\Local\Temp\ipykernel_14400\820435583.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [7]:
df.drop('wd', axis=1, inplace=True)

In [8]:

df_features = tsfresh.extract_features(df, column_id='station', column_sort='timestamp', 
                                       default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters())
df_features.columns


Feature Extraction: 100%|██████████| 20/20 [00:08<00:00,  2.49it/s]


Index(['year__sum_values', 'year__median', 'year__mean', 'year__length',
       'year__standard_deviation', 'year__variance', 'year__root_mean_square',
       'year__maximum', 'year__absolute_maximum', 'year__minimum',
       ...
       'WSPM__sum_values', 'WSPM__median', 'WSPM__mean', 'WSPM__length',
       'WSPM__standard_deviation', 'WSPM__variance', 'WSPM__root_mean_square',
       'WSPM__maximum', 'WSPM__absolute_maximum', 'WSPM__minimum'],
      dtype='object', length=150)

In [10]:
df_features


,year__sum_values,year__median,year__mean,year__length,year__standard_deviation,year__variance,year__root_mean_square,year__maximum,year__absolute_maximum,year__minimum,...,WSPM__sum_values,WSPM__median,WSPM__mean,WSPM__length,WSPM__standard_deviation,WSPM__variance,WSPM__root_mean_square,WSPM__maximum,WSPM__absolute_maximum,WSPM__minimum
Aotizhongxin,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,59907.015948,1.4,1.708505,35064.0,1.203814,1.449167,2.090013,11.2,11.2,0.0
Changping,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,64997.577554,1.5,1.853684,35064.0,1.308993,1.713464,2.269275,10.0,10.0,0.0
Dingling,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,64997.577554,1.5,1.853684,35064.0,1.308993,1.713464,2.269275,10.0,10.0,0.0
Dongsi,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,65244.715948,1.5,1.860732,35064.0,1.280097,1.638648,2.258533,10.5,10.5,0.0
Guanyuan,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,59907.015948,1.4,1.708505,35064.0,1.203814,1.449167,2.090013,11.2,11.2,0.0
Gucheng,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,47118.047843,1.0,1.343773,35064.0,1.150436,1.323503,1.768962,12.0,12.0,0.0
Huairou,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,57930.255817,1.3,1.652129,35064.0,1.198291,1.435902,2.040939,12.9,12.9,0.0
Nongzhanguan,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,65244.715948,1.5,1.860732,35064.0,1.280097,1.638648,2.258533,10.5,10.5,0.0
Shunyi,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,63375.907264,1.5,1.807435,35064.0,1.286993,1.656352,2.218823,12.8,12.8,0.0
Tiantan,70642128.0,2015.0,2014.66256,35064.0,1.177197,1.385792,2014.662904,2017.0,2017.0,2013.0,...,65244.715948,1.5,1.860732,35064.0,1.280097,1.638648,2.258533,10.5,10.5,0.0


In [17]:
df_features.RAIN__mean

Aotizhongxin     0.067419
Changping        0.060372
Dingling         0.060372
Dongsi           0.064020
Guanyuan         0.067419
Gucheng          0.064453
Huairou          0.067934
Nongzhanguan     0.064020
Shunyi           0.061099
Tiantan          0.064020
Wanliu           0.068261
Wanshouxigong    0.064320
Name: RAIN__mean, dtype: float64